In [1]:
import pickle
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
%matplotlib inline 



In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

In [3]:
# Detect device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
def save_data(fp, data):
    with open(fp, "wb") as fo:
        pickle.dump(data, fo)

def load_data(fp):
    with open(fp, "rb") as fo:
        data = pickle.load(fo)
    return np.array(data)

In [5]:
def stack_folder_files(fp):
    data = []
    list_dir = os.listdir(fp) 
    list_dir.sort()
    for f in list_dir:
        if not f.startswith('.'):
            data.extend(load_data(fp+f))

    return np.array(data)

In [6]:
fp = "./data/"

In [7]:
data = stack_folder_files(fp)

/home/wangzhi/anaconda3/envs/cosim/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [8]:
len(data)

7898

In [9]:
x = []
y = []

for i in data:
    x.append(i[0])
    encoded_arr = np.zeros((1, 2), dtype=int)
    encoded_arr[0,i[1]] = 1 
    y.append(i[1])

In [10]:
# one-hot encoding 0-> [1,0], 1->[0,1]
y_one_hot = np.eye(2)[y]
y_one_hot

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(np.array(x).astype(np.float32), np.array(y_one_hot).astype(np.float32), test_size=0.2, random_state=42)

# Convert to PyTorch tensors and move to device later
X_train_tensor = torch.tensor(X_train)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float)
X_test_tensor = torch.tensor(X_test)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.float)


# Create DataLoaders
train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=64)

In [12]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(26, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = F.softmax(x, dim=1)
        return x

In [13]:
model = BinaryClassifier().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 50

for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for batch_x, batch_y in train_loader:
        # Move data to device
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device).float()  # one-hot labels must be float

        # Forward, backward, optimize
        optimizer.zero_grad()
        outputs = model(batch_x)              # logits: [batch_size, 2]
        loss = criterion(outputs, batch_y)    # one-hot: [batch_size, 2]
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device).float()  # one-hot, so keep as float

        outputs = model(batch_x)
        predicted = torch.argmax(outputs, dim=1)         # logits → class index
        true_labels = torch.argmax(batch_y, dim=1)       # one-hot → class index

        correct += (predicted == true_labels).sum().item()
        total += batch_y.size(0)

print(f"Test Accuracy: {100 * correct / total:.2f}%")


In [ ]:
torch.save(model.state_dict(), 'binary_classifier_0515.pth')
print("Model saved to binary_classifier_0515.pth")

In [15]:
# Load model later
# Recreate the model architecture
model = BinaryClassifier().to(device)

# Load the saved weights
model.load_state_dict(torch.load('binary_classifier.pth'))

# Set to evaluation mode
model.eval()
print("Model loaded from binary_classifier.pth")

Model loaded from binary_classifier.pth


In [16]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device).float()  # one-hot, so keep as float

        outputs = model(batch_x)
        predicted = torch.argmax(outputs, dim=1)         # logits → class index
        true_labels = torch.argmax(batch_y, dim=1)       # one-hot → class index

        correct += (predicted == true_labels).sum().item()
        total += batch_y.size(0)

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 96.14%


## Regression model for I-SIG fitting

In [ ]:
fp = "./data_isig_mpr5/"

In [ ]:
data = stack_folder_files(fp)
len(data)

In [ ]:
x = []
y = []
phases = []

for i in data:
    x.append(i[0])
    y.append(i[1]/10)
    phases.append(i[2])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RegressionNet(nn.Module):
    def __init__(self):
        super(RegressionNet, self).__init__()
        self.fc1 = nn.Linear(26, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 1)  # Output a single value

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = torch.sigmoid(self.fc5(x))  # Output in range (0, 1)
        x = x * 40  # Scale to range (0, 50)
        return x


In [ ]:
class ImprovedRegressionNet(nn.Module):
    def __init__(self):
        super(ImprovedRegressionNet, self).__init__()
        self.fc1 = nn.Linear(26, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.fc2 = nn.Linear(64, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, 32)
        self.bn4 = nn.BatchNorm1d(32)
        self.output = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.fc1(x)))
        x = self.dropout(F.leaky_relu(self.bn2(self.fc2(x))))
        x = self.dropout(F.leaky_relu(self.bn3(self.fc3(x))))
        x = F.leaky_relu(self.bn4(self.fc4(x)))
        x = torch.sigmoid(self.output(x)) * 40
        return x


In [ ]:
class ResNetRegression(nn.Module):
    def __init__(self):
        super(ResNetRegression, self).__init__()
        self.fc1 = nn.Linear(26, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.output = nn.Linear(64, 1)

    def forward(self, x):
        x1 = F.relu(self.fc1(x))
        x2 = F.relu(self.fc2(x1)) + x1  # residual connection
        x3 = F.relu(self.fc3(x2)) + x2  # another residual connection
        out = torch.sigmoid(self.output(x3)) * 40
        return out


In [ ]:
class CNNRegressionNet(nn.Module):
    def __init__(self):
        super(CNNRegressionNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(16, 1)

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch_size, 1, 26)
        x = F.relu(self.conv1(x))  # (batch_size, 16, 26)
        x = self.pool(x).squeeze(-1)  # (batch_size, 16)
        x = torch.sigmoid(self.fc(x)) * 40
        return x


In [ ]:
import torch
import torch.nn as nn

class TransformerRegressionNet(nn.Module):
    def __init__(self, input_dim=26, d_model=64, nhead=4, num_layers=2, dim_feedforward=128, dropout=0.1):
        super(TransformerRegressionNet, self).__init__()
        
        self.input_dim = input_dim
        self.d_model = d_model
        
        # Project input features to d_model
        self.input_proj = nn.Linear(input_dim, d_model)
        
        # Positional encoding: learnable (can also use sinusoidal)
        self.pos_embedding = nn.Parameter(torch.randn(1, input_dim, d_model))
        
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout, activation='relu'
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Pool and predict
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.regressor = nn.Sequential(
            nn.Linear(d_model, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()  # output in (0,1)
        )

    def forward(self, x):
        # x shape: (batch_size, 26)
        x = self.input_proj(x)  # (batch_size, d_model)
        x = x.unsqueeze(1)  # (batch_size, 1, d_model)
        x = x + self.pos_embedding[:, :x.size(1), :]  # Add position info
        
        # Transformer expects (seq_len, batch, d_model), so permute
        x = x.permute(1, 0, 2)
        x = self.transformer_encoder(x)  # (seq_len, batch, d_model)
        x = x.permute(1, 2, 0)  # (batch, d_model, seq_len)
        
        # Pool across sequence (dim=2), output (batch, d_model)
        x = self.pool(x).squeeze(-1)
        
        # Predict
        out = self.regressor(x) * 40  # Scale to [0, 50]
        return out


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(np.array(x).astype(np.float32), np.array(y).astype(np.float32), test_size=0.2, random_state=42)

# Convert to PyTorch tensors and move to device later
X_train_tensor = torch.tensor(X_train)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float)
X_test_tensor = torch.tensor(X_test)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.float)


# Create DataLoaders
train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=64)

In [ ]:
model = RegressionNet().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 300

for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for batch_x, batch_y in train_loader:
        # Move data to device
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device).float()  # one-hot labels must be float

        # Forward, backward, optimize
        optimizer.zero_grad()
        outputs = model(batch_x)              # logits: [batch_size, 2]
        loss = criterion(outputs, batch_y.unsqueeze(1))    # one-hot: [batch_size, 2]
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")
    


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

model.to(device)
model.eval()

predictions = []
targets = []

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        outputs = model(batch_x).squeeze()
        predictions.extend(outputs.cpu().numpy())  # move to CPU before converting to numpy
        targets.extend(batch_y.cpu().numpy())

# Convert to numpy arrays
predictions = np.array(predictions)
targets = np.array(targets)

# Compute evaluation metrics
mse = mean_squared_error(targets, predictions)
mae = mean_absolute_error(targets, predictions)
r2 = r2_score(targets, predictions)


In [ ]:
print("mse,mae,r2 ",mse,mae,r2 )

In [ ]:
torch.save(model.state_dict(), 'RegressionNet_0520_pr5.pth')
print("Model saved to RegressionNet_0520_pr5.pth")

## Convert Regression to Classification

In [7]:
fp = "./data_isig_mpr5/"

In [29]:
data = stack_folder_files(fp)
len(data)

/home/wangzhi/anaconda3/envs/cosim/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


4431

In [30]:
x = []
y = []
phases = []

for i in data:
    x.append(i[0])  # assuming i[0] is a list or NumPy array of 26 features

    # Convert i[1] to a class in {0, 1, ..., 8}, assuming i[1] ∈ [0, 40]
    class_index = int((i[1]/10) // 5)  # safely map 0→0, 5→1, ..., 40→8
    y_onehot = torch.nn.functional.one_hot(torch.tensor(class_index), num_classes=9).float()
    y.append(y_onehot)

    phases.append(i[2])  # store i[2] as-is

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RegressionNet(nn.Module):
    def __init__(self):
        super(RegressionNet, self).__init__()
        self.fc1 = nn.Linear(26, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 9)   # Output a single value

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        logits = self.output(x)
        probabilities = F.softmax(logits, dim=1) 
        return probabilities

In [32]:
import torch
from sklearn.model_selection import train_test_split

# Convert x and y into tensors
X = torch.tensor(x, dtype=torch.float32)                  # shape: [num_samples, 26]
Y = torch.stack(y).float()                                # shape: [num_samples, 9]

# Convert to NumPy for train_test_split
X_np = X.numpy()
Y_np = Y.numpy()

# Train/test split
X_train, X_test, Y_train, Y_test = train_test_split(X_np, Y_np, test_size=0.2, random_state=42)

# Convert back to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32)


In [37]:
# Create DataLoaders
train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=64)

In [38]:
model = RegressionNet().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [39]:
epochs = 200

for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for batch_x, batch_y in train_loader:
        # Move data to device
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device).float()  # one-hot labels must be float

        # Forward, backward, optimize
        optimizer.zero_grad()
        outputs = model(batch_x)              # logits: [batch_size, 2]
        loss = criterion(outputs, batch_y)    # one-hot: [batch_size, 2]
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")


Epoch 1/200, Loss: 16.5540
Epoch 2/200, Loss: 12.5960
Epoch 3/200, Loss: 11.0430
Epoch 4/200, Loss: 9.8742
Epoch 5/200, Loss: 9.0382
Epoch 6/200, Loss: 8.6236
Epoch 7/200, Loss: 8.1087
Epoch 8/200, Loss: 7.7694
Epoch 9/200, Loss: 7.4808
Epoch 10/200, Loss: 7.1667
Epoch 11/200, Loss: 6.9760
Epoch 12/200, Loss: 6.7843
Epoch 13/200, Loss: 6.6723
Epoch 14/200, Loss: 6.5165
Epoch 15/200, Loss: 6.3705
Epoch 16/200, Loss: 6.3731
Epoch 17/200, Loss: 6.2236
Epoch 18/200, Loss: 6.1688
Epoch 19/200, Loss: 6.0282
Epoch 20/200, Loss: 6.0424
Epoch 21/200, Loss: 5.9799
Epoch 22/200, Loss: 5.8796
Epoch 23/200, Loss: 5.9734
Epoch 24/200, Loss: 5.9373
Epoch 25/200, Loss: 5.7922
Epoch 26/200, Loss: 5.7199
Epoch 27/200, Loss: 5.8622
Epoch 28/200, Loss: 5.7453
Epoch 29/200, Loss: 5.8129
Epoch 30/200, Loss: 5.5989
Epoch 31/200, Loss: 5.5877
Epoch 32/200, Loss: 5.5809
Epoch 33/200, Loss: 5.5059
Epoch 34/200, Loss: 5.4854
Epoch 35/200, Loss: 5.5523
Epoch 36/200, Loss: 5.4399
Epoch 37/200, Loss: 5.4781
Epoch 3

In [40]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device).float()  # one-hot, so keep as float

        outputs = model(batch_x)
        predicted = torch.argmax(outputs, dim=1)         # logits → class index
        true_labels = torch.argmax(batch_y, dim=1)       # one-hot → class index

        correct += (predicted == true_labels).sum().item()
        total += batch_y.size(0)

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 80.95%


In [41]:
torch.save(model.state_dict(), 'ISIG_surr_class_0522.pth')
print("Model saved to ISIG_surr_class_0522.pth")

Model saved to ISIG_surr_class_0522.pth
